In [ ]:
import os
import sys
import random
import pathlib

# processing
import geojson
import utm
import pyproj
import shapefile
import shapely
import shapely.ops
import shapely.geometry

# interactive/visualization
import folium
from tqdm.notebook import tqdm
from IPython.display import display

shapefile: `forestarea_2562.rar`

contains the _entire_ country's forest regions, somehow condensed as 4 multipolygon records

Using UTM Zone 47N projection instead of WGS84 (latlon)

In [ ]:
# read the shapefile
# stole from https://gist.github.com/frankrowe/6071443

# set path here
SHAPEFILE_PATH=r"D:\forestarea_2562\forestarea62_wgs84_z47.shp"
shpfile = str(pathlib.Path(SHAPEFILE_PATH).resolve())

reader = shapefile.Reader(shpfile)
fields = reader.fields[1:]
field_names = [field[0] for field in fields]

print(reader)

In [ ]:
# for some fucking reason new shapefile contains 4 multipolygons
# instead of flat collection of shapes
# different processing required
fshapes = []
for shape in reader.shapes():
    fshapes.append(shapely.geometry.shape(shape))


In [ ]:
total_shape = 0
for f in fshapes:
    flen = len(f)
    total_shape += flen
    print(flen)
print(' '.join(["total:", str(total_shape)]))

In [ ]:
# WGS84 latlng
wgs84 = pyproj.Proj(init="epsg:4326") 
# UTM 47N
utm47n = pyproj.Proj(init="epsg:32647") 

proj_latlng2utm = pyproj.Transformer.from_proj(wgs84, utm47n)
proj_utm2latlng = pyproj.Transformer.from_proj(utm47n, wgs84)

# _ป่าควนเคร็ง~~อยู่ไหนวะสัส~~ GET !!!_

_ไม่ใช่ก็ใกล้เคียงละวะ_

-----

bounding box หยาบๆ

    (1)----(2)
     |      |
    (5)     |
      \     |
       (4)-(3)

    latlon
    1) 8.132766,99.977713
    2) 8.129368,100.220099
    3) 7.713491,100.225925
    4) 7.716212,100.123490
    5) 7.849215,100.044182

In [ ]:
# DO NOT FUCKING DO IT
#allgon = shapely.ops.unary_union(fshapes)

forest0 = fshapes[0]
kuankreng = shapely.geometry.Polygon([
    (99.977710,8.132766), (100.220099,8.129368),
    (100.225925,7.713491), (100.123490,7.716212), (100.044182,7.849215)]
)
kuankreng_utm = shapely.ops.transform(proj_latlng2utm.transform, kuankreng)

roi62 = []
for shape in tqdm(forest0):
    if kuankreng_utm.contains(shape):
        roi62.append(shape)
display(kuankreng_utm)


In [ ]:
kuankreng62 = shapely.ops.unary_union(roi62)
kuankreng62_wgs84 = shapely.ops.transform(proj_utm2latlng.transform, kuankreng62)
display(kuankreng62)

In [ ]:
smap = folium.Mapsmap = folium.Map(location=[7.89, 100.10], zoom_start=6, tiles="Stamen Terrain")

folium.GeoJson(kuankreng62_wgs84.__geo_interface__, name="kuankreng").add_to(smap)

kuankreng_geo = shapely.geometry.mapping(kuankreng)
folium.GeoJson(kuankreng_geo, name="est. bbox").add_to(smap)

folium.TileLayer(tiles="cartodb positron").add_to(smap)
folium.LayerControl().add_to(smap)

display(smap)

In [ ]:
with open(str(pathlib.Path("./kuankreng62.geojson").resolve()), 'w') as k62f:
    k62f.write(str(kuankreng62_wgs84.__geo_interface__))